In [46]:
import pandas as pd
import numpy as np
import re

# 1. Load data
df = pd.read_csv("glassdoor_jobs.csv")

# ---------------------------
# 2. Salary Estimate 전처리
# ---------------------------

def extract_salary(s):
    if pd.isnull(s) or '-1' in s:
        return np.nan, np.nan, np.nan

    s = s.lower()
    s = s.replace('employer provided salary:', '')
    s = s.replace('per hour', '')
    s = re.sub(r'\(.*?\)', '', s)  # 괄호 안 제거
    s = s.replace('$', '').replace('k', '').strip()

    try:
        min_sal, max_sal = map(int, s.split('-'))
        avg_sal = (min_sal + max_sal) / 2
        return min_sal, max_sal, avg_sal
    except:
        return np.nan, np.nan, np.nan

# Apply to Salary Estimate
df[["min_salary", "max_salary", "avg_salary"]] = df["Salary Estimate"].apply(
    lambda x: pd.Series(extract_salary(x))
)

# ---------------------------
# 3. Size 전처리 (원하는 그룹 기준 적용)
# ---------------------------

def map_size(size_str):
    if pd.isnull(size_str) or size_str in ['-1', 'Unknown']:
        return np.nan
    size_str = size_str.strip()
    if size_str in ['0 to 50 employees', '51 to 200 employees']:
        return 'Small'
    elif size_str in ['201 to 500 employees', '501 to 1000 employees']:
        return 'Medium'
    elif size_str in ['1001 to 5000 employees', '5001 to 10000 employees']:
        return 'Large'
    elif size_str == '10000+ employees':
        return 'Very Large'
    else:
        return np.nan

df["Size_cleaned"] = df["Size"].apply(map_size)

# ---------------------------
# 4. Founded → 회사 나이 계산
# ---------------------------

df["Founded_cleaned"] = df["Founded"].apply(lambda x: x if x > 1800 else np.nan)
df["Company_age"] = df["Founded_cleaned"].apply(lambda x: 2025 - x if pd.notnull(x) else np.nan)


# ---------------------------
# 결과 확인
# ---------------------------
# 모든 행과 열을 다 보여주도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 이제 전체 출력이 가능
print(df[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary',
          'Size', 'Size_cleaned',
          'Founded', 'Founded_cleaned', 'Company_age']])

# ---------------------------
# 결측치 전 상태 저장 및 출력
# ---------------------------
print(" 결측치 처리 전 상태:")
missing_before = df[['avg_salary', 'Size_cleaned', 'Company_age', 'Founded']].isnull().sum()
print(missing_before)





                               Salary Estimate  min_salary  max_salary  \
0                   $53K-$91K (Glassdoor est.)        53.0        91.0   
1                  $63K-$112K (Glassdoor est.)        63.0       112.0   
2                   $80K-$90K (Glassdoor est.)        80.0        90.0   
3                   $56K-$97K (Glassdoor est.)        56.0        97.0   
4                  $86K-$143K (Glassdoor est.)        86.0       143.0   
5                  $71K-$119K (Glassdoor est.)        71.0       119.0   
6                   $54K-$93K (Glassdoor est.)        54.0        93.0   
7                  $86K-$142K (Glassdoor est.)        86.0       142.0   
8                   $38K-$84K (Glassdoor est.)        38.0        84.0   
9                 $120K-$160K (Glassdoor est.)       120.0       160.0   
10                $126K-$201K (Glassdoor est.)       126.0       201.0   
11                 $64K-$106K (Glassdoor est.)        64.0       106.0   
12                $106K-$172K (Glassdo

In [47]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드
df = pd.read_csv("glassdoor_jobs.csv")

# 2. Salary Estimate 파싱 함수
def extract_salary(s):
    if pd.isnull(s) or s == '-1' or '-1' in str(s):
        return np.nan, np.nan, np.nan
    s = str(s).lower()
    s = s.replace('employer provided salary:', '').replace('per hour', '')
    s = re.sub(r'\(.*?\)', '', s).replace('$', '').replace('k', '').strip()
    try:
        min_sal, max_sal = map(int, s.split('-'))
        avg_sal = (min_sal + max_sal) / 2
        return min_sal, max_sal, avg_sal
    except:
        return np.nan, np.nan, np.nan

# 3. Salary 컬럼 처리
df['Salary Estimate'] = df['Salary Estimate'].replace('-1', np.nan)
df[['min_salary', 'max_salary', 'avg_salary']] = df['Salary Estimate'].apply(lambda x: pd.Series(extract_salary(x)))

# 4. Size 전처리 및 그룹화
df['Size'] = df['Size'].replace(['-1', 'Unknown'], np.nan)

def map_size(size_str):
    if pd.isnull(size_str):
        return np.nan
    size_str = size_str.strip()
    if size_str in ['0 to 50 employees', '51 to 200 employees']:
        return 'Small'
    elif size_str in ['201 to 500 employees', '501 to 1000 employees']:
        return 'Medium'
    elif size_str in ['1001 to 5000 employees', '5001 to 10000 employees']:
        return 'Large'
    elif size_str == '10000+ employees':
        return 'Very Large'
    return np.nan

df['Size_cleaned'] = df['Size'].apply(map_size)

# 5. Founded → Company_age 생성 및 같은 회사 기준 보완
df['Founded'] = pd.to_numeric(df['Founded'], errors='coerce')
df['Founded'] = df['Founded'].replace(-1, np.nan)
if 'Company Name' in df.columns:
    df['Founded'] = df.groupby('Company Name')['Founded'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

df['Founded_cleaned'] = df['Founded']
df['Company_age'] = df['Founded_cleaned'].apply(lambda x: 2025 - x if pd.notnull(x) and x > 1800 else np.nan)

# 6. 그룹 평균 계산
group_salary = df.groupby('Size_cleaned')[['min_salary', 'max_salary', 'avg_salary']].mean()
company_age_mean = df.groupby('Company Name')['Company_age'].mean()

# 7. 보완 함수
def fill_features(row):
    size = row['Size_cleaned']
    company = row['Company Name']
    if pd.isnull(row['min_salary']) and size in group_salary.index:
        row['min_salary'] = group_salary.loc[size, 'min_salary']
    if pd.isnull(row['max_salary']) and size in group_salary.index:
        row['max_salary'] = group_salary.loc[size, 'max_salary']
    if pd.isnull(row['avg_salary']) and size in group_salary.index:
        row['avg_salary'] = group_salary.loc[size, 'avg_salary']
    if pd.isnull(row['Company_age']) and company in company_age_mean.index:
        row['Company_age'] = company_age_mean[company]
    return row

df = df.apply(fill_features, axis=1)

# 8. 남은 결측치는 전체 평균/최빈값으로 대체
df['min_salary'] = df['min_salary'].fillna(round(df['min_salary'].mean()))
df['max_salary'] = df['max_salary'].fillna(round(df['max_salary'].mean()))
df['avg_salary'] = df['avg_salary'].fillna(round(df['avg_salary'].mean()))
df['Company_age'] = df['Company_age'].fillna(round(df['Company_age'].mean()))
df['Founded_cleaned'] = df['Founded_cleaned'].fillna(round(df['Founded_cleaned'].mean()))
df['Size_cleaned'] = df['Size_cleaned'].fillna(df['Size_cleaned'].mode()[0])
df['Size'] = df['Size'].fillna(df['Size'].mode()[0])


# 9. Salary Estimate 재구성
df['Salary Estimate'] = df.apply(lambda row: f"${int(row['min_salary'])}K-${int(row['max_salary'])}K (Glassdoor est.)", axis=1)

# -- 전처리 후 결측치 상태 출력 --
print("== 결측치 개수 현황 (전처리 후) ==")
print(df[['min_salary', 'max_salary', 'avg_salary', 'Size_cleaned', 'Founded_cleaned', 'Company_age']].isnull().sum())
print("\n")

# 10. pandas 전체 행 출력 옵션 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 11. 전체 결과 출력
output_cols = ['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary',
               'Size', 'Size_cleaned', 'Founded', 'Founded_cleaned', 'Company_age']

print(df[output_cols])








== 결측치 개수 현황 (전처리 후) ==
min_salary         0
max_salary         0
avg_salary         0
Size_cleaned       0
Founded_cleaned    0
Company_age        0
dtype: int64


                  Salary Estimate  min_salary  max_salary  avg_salary  \
0      $53K-$91K (Glassdoor est.)   53.000000   91.000000   72.000000   
1     $63K-$112K (Glassdoor est.)   63.000000  112.000000   87.500000   
2      $80K-$90K (Glassdoor est.)   80.000000   90.000000   85.000000   
3      $56K-$97K (Glassdoor est.)   56.000000   97.000000   76.500000   
4     $86K-$143K (Glassdoor est.)   86.000000  143.000000  114.500000   
5     $71K-$119K (Glassdoor est.)   71.000000  119.000000   95.000000   
6      $54K-$93K (Glassdoor est.)   54.000000   93.000000   73.500000   
7     $86K-$142K (Glassdoor est.)   86.000000  142.000000  114.000000   
8      $38K-$84K (Glassdoor est.)   38.000000   84.000000   61.000000   
9    $120K-$160K (Glassdoor est.)  120.000000  160.000000  140.000000   
10   $126K-$201K (Glassdoor est.

C:\Users\User\AppData\Local\Temp\ipykernel_11676\2538307848.py:49: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Founded'] = df.groupby('Company Name')['Founded'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
